# 작업 1유형

## 데이터 로드

In [71]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')

## 성별이 Male인 환자들의 age 평균값

In [72]:
df.age = df.age.str.replace('*','')
df.age = df.age.astype(int)
print(df.loc[df['gender'] == 'Male'].age.mean())

44.68623481781376


## bmi 컬럼의 결측치를 bmi 컬럼의 결측치를 제외한 나머지 값들의 중앙값으로 채웠을 경우 bmi 컬럼의 평균(소숫점 3자리까지)

In [73]:
print(round(df.bmi.fillna(df.bmi.median()).mean(),3))

29.166


## bmi 컬럼의 결측치를 이전 행의 bmi 값으로 채웠을 경우 bmi 컬럼의 평균(소숫점 3자리까지)

In [74]:
print(round(df.bmi.fillna(method='ffill').mean(),3))

29.188


/tmp/ipykernel_50006/2022984429.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  print(round(df.bmi.fillna(method='ffill').mean(),3))


## bmi 컬럼의 각 결측치들을 결측치를 가진 환자 나이대의 평균 bmi 값으로 대체한 후 대체된 bmi 컬럼의 평균(소숫점 3자리까지)

In [75]:
df['age_group'] = df['age'].map(lambda x : x//10 * 10)
print(round(df.bmi.fillna(df.groupby('age_group').bmi.mean()).mean(),3))

29.198


## avg_glucose_level 컬럼의 값이 200 이상인 데이터를 모두 199로 변경하고 stroke값이 1인 데이터의 avg_glucose_level값의 평균(소숫점 3자리까지)

In [77]:
df.loc[df['avg_glucose_level'] >= 200].avg_glucose_level = 199
print(round(df.loc[df.stroke == 1].avg_glucose_level.mean(),3))

131.428


/tmp/ipykernel_50006/4003914888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['avg_glucose_level'] >= 200].avg_glucose_level = 199


## 데이터 로드

In [78]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


## Attack 컬럼의 값을 기준으로 내림차순 정렬했을 때 상위 400위 까지 포켓몬들과 401-800위가지의 포켓몬들 사이에서 전설포켓몬의 숫자 차이

In [92]:
print(len(df.sort_values('Attack',ascending=False)[:400].loc[df['Legendary']==True]) - len(df.sort_values('Attack',ascending=False)[400:].loc[df['Legendary']==True]))

57


## Type 1 컬럼의 종류에 따른 Total 컬럼의 평균값을 내림차순 정렬했을 때 상위 3번째 Type 1

In [96]:
print(df.groupby('Type 1').Total.mean().sort_values(ascending=False).index[2])

Flying


## 결측치가 존재하는 행을 모두 지운 후 처음부터 60% 데이터를 추출하여 Defense 컬럼의 1분위수

In [104]:
print(df.dropna()[:int(len(df.dropna())*0.6)].Defense.describe()['25%'])

50.0


## Type 1 컬럼의 속성이 Fire 인 포켓몬들의 Attack의 평균이상인 Water 속성의 포켓몬 수

In [108]:
print(len(df.loc[(df['Type 1'] == 'Water')& (df['Attack'] >= df.loc[df['Type 1']=='Fire'].Attack.mean())]))

37


## 각 세대 중의 Speed와 Defense 컬럼의 차이(절댓값)이 가장 큰 세대

In [116]:
print(abs(df.groupby('Generation').Speed.mean() - df.groupby('Generation').Defense.mean()).sort_values(ascending=False).index[0])

2


# 작업 2유형

In [143]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score 
from sklearn.ensemble import RandomForestClassifier 

train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/test.csv')
train['age'] = train['age'].str.replace('*','')
train['age'] = train['age'].astype(int)
test['age'] = test['age'].astype(int)
x = train.drop(['id','stroke'],axis=1)
y = train['stroke']
z = test.drop(['id'],axis=1)

for i in x.describe(include='object').columns:
    x[i] = LabelEncoder().fit_transform(x[i])
    z[i] = LabelEncoder().fit_transform(z[i])

x.bmi.fillna(x.bmi.mean(),inplace=True)
print(x.describe(include='all'))

xtr,xte,ytr,yte = train_test_split(x,y,test_size=0.33,random_state = 23)

RFC = RandomForestClassifier(random_state = 23)
RFC.fit(xtr,ytr)

pred = RFC.predict(xte)
pred_proba = RFC.predict_proba(xte)[:,1]

acc = accuracy_score(yte,pred)
f1 = f1_score(yte,pred)
precision = precision_score(yte,pred)
recall = recall_score(yte,pred)
auc = roc_auc_score(yte,pred_proba)
print(acc)
print(f1)
print(precision)
print(recall)
print(auc)

test['stroke'] = RFC.predict(z)
print(test.describe(include='all'))

            gender          age  hypertension  heart_disease  ever_married  \
count  1137.000000  1137.000000   1137.000000    1137.000000   1137.000000   
mean      0.436236    45.189094      0.118734       0.068602      0.676341   
std       0.497907    23.070207      0.323617       0.252887      0.468077   
min       0.000000     0.000000      0.000000       0.000000      0.000000   
25%       0.000000    28.000000      0.000000       0.000000      0.000000   
50%       0.000000    48.000000      0.000000       0.000000      1.000000   
75%       1.000000    63.000000      0.000000       0.000000      1.000000   
max       2.000000    82.000000      1.000000       1.000000      1.000000   

        work_type  Residence_type  avg_glucose_level          bmi  \
count  1137.00000     1137.000000        1137.000000  1137.000000   
mean      2.16007        0.516271         107.664002    29.198065   
std       1.07093        0.499955          47.618723     7.492022   
min       0.00000    

/tmp/ipykernel_50006/1761997069.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x.bmi.fillna(x.bmi.mean(),inplace=True)
